In [2]:
import urllib.request
import bs4
import pandas as pd

path = "players_link.txt"

with open(path) as f:
    links = f.readlines()
    
data_frame = pd.DataFrame()
    
for index, link in enumerate(links):
    link = link.replace('\n','')
    url = 'http://en.espn.co.uk/' + link
    soup = bs4.BeautifulSoup(urllib.request.urlopen(url).read(), "lxml")

    print(str(index) + ":" + link)

    data = soup.find("div", class_="scrumPlayerName")

    stats = {}
    stats["name"] = data.string

    data = soup.find_all("div", class_="scrumPlayerDesc")
    for item in data:
        try:
            key = item.find("b").string
            text = str(item)
            start_index = text.find("</b>")
            end_index = text.find("</div>")
            value = text[start_index + 4:end_index].strip()
            stats[key] = value
        except AttributeError:
            pass

    data = soup.find("thead").find_all("th")
    head =[]
    for item in data:
        head.append(item.string)


    data = soup.find("table", class_="engineTable").find_all("tr")
    for item in data:
        if item.find("b").string == "All Tests":
            for index, table_data in enumerate(item.find_all("td")):
                if index > 0:
                    stats[head[index]] =  table_data.string
                    
    for column in stats.keys():
        if not column in data_frame.columns:
            data_frame[column] = 0               
        
    values = []
    for v in stats.values():
        values.append(v)

        
    series = pd.Series(values, index=stats.keys())
    data_frame = data_frame.append(series, ignore_index = True)
    

0:/newzealand/rugby/player/1553.html
1:/newzealand/rugby/player/1555.html
2:/newzealand/rugby/player/1556.html
3:/newzealand/rugby/player/1558.html
4:/newzealand/rugby/player/1559.html
5:/newzealand/rugby/player/1562.html
6:/newzealand/rugby/player/1564.html
7:/newzealand/rugby/player/1566.html
8:/newzealand/rugby/player/1567.html
9:/newzealand/rugby/player/1568.html
10:/newzealand/rugby/player/139498.html
11:/newzealand/rugby/player/1574.html
12:/newzealand/rugby/player/1575.html
13:/newzealand/rugby/player/1576.html
14:/newzealand/rugby/player/1579.html
15:/newzealand/rugby/player/1678.html
16:/newzealand/rugby/player/1680.html
17:/newzealand/rugby/player/1681.html
18:/newzealand/rugby/player/1682.html
19:/newzealand/rugby/player/1683.html
20:/newzealand/rugby/player/1685.html
21:/newzealand/rugby/player/1686.html
22:/newzealand/rugby/player/1716.html
23:/newzealand/rugby/player/1717.html
24:/newzealand/rugby/player/1719.html
25:/newzealand/rugby/player/1721.html
26:/newzealand/rugby

In [25]:
data_frame.head()

,name,Full name,Born,Died,Major teams,Position,Span,Mat,Start,Sub,...,Lost,Draw,%,Other,Relations,Also known as,Height,Weight,Current age,Nickname
0,Opai Asher,Albert Arapeha Paurini Asher,"December 3, 1879, Tauranga","January 8, 1965, Auckland (aged 85 years 36 days)",New Zealand,Wing,1903-1903,1,1,0,...,0,0,100.00,0,0,0,0,0,0,0
1,Reuben Cooke,Reuben James Cooke,"circa 1878, Catlins","May 10, 1940, Melbourne",New Zealand,Lock,1903-1903,1,1,0,...,0,0,100.00,0,0,0,0,0,0,0
2,Jimmy Duncan,James Duncan,"November 12, 1869, Dunedin","October 19, 1953, Dunedin (aged 83 years 341 d...",New Zealand,Fly-half,1903-1903,1,1,0,...,0,0,100.00,Referee,0,0,0,0,0,0
3,Bernard Fanning,Bernard John Fanning,"November 11, 1874, Christchurch","July 9, 1946, Christchurch (aged 71 years 240 ...",New Zealand,Lock,1903-1904,2,2,0,...,0,0,100.00,NaN,0,0,0,0,0,0
4,David Gallaher,David Gallaher,"October 30, 1873, Ramelton, Ulster","October 4, 1917, Gravenstafel (aged 43 years 3...",New Zealand,Wing-forward,1903-1906,6,6,0,...,1,0,83.33,NaN,0,0,0,0,0,0


In [26]:
data_frame.describe()

,name,Full name,Born,Died,Major teams,Position,Span,Mat,Start,Sub,...,Lost,Draw,%,Other,Relations,Also known as,Height,Weight,Current age,Nickname
count,862,862,862,398,862,862,862,862,862,862,...,862,862,862,7,46,167,511,511,712,366
unique,860,861,861,398,237,23,399,83,75,32,...,20,5,181,3,10,2,16,80,461,5
top,Mark Robinson,Thomas William Lynch,"December 15, 1959, Auckland","August 2, 2017, Auckland (aged 82 years 213 days)",New Zealand,Prop,1913-1913,1,1,0,...,0,0,100.00,Referee,0,0,0,0,0,0
freq,2,2,2,1,420,114,20,163,154,594,...,268,618,230,4,37,166,219,219,248,362


In [3]:
data_frame.to_csv( 'player_stats.csv' )